# File txt di input (ottenibile dall'app di WhatsApp, senza esportare i media)

In [ ]:
filename = "chat.txt"

# Struttura base

In [ ]:
import os
import numpy as np
import pandas as pd
from datetime import datetime
import re
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, Markdown, Latex
import operator
import sys
%matplotlib inline  

class ChatLine:
    """
        Modello di linea di chat
    """
    messaggiWhatsapp = [
        "ti ha aggiunto",
        "ha aggiunto",
        "Hai aggiunto",
        "ha abbandonato",
        "Hai abbandonato",
        "ha rimosso",
        "ha cambiato",
        "Hai cambiato",
        "I messaggi inviati a questo gruppo sono ora protetti con la crittografia end-to-end.",
        "è cambiato. Tocca per maggiori info.",
        "ti ha rimosso",
        "Hai eliminato l'immagine del gruppo",
        "ha cambiato numero da ",
        "I messaggi che invii in questa chat",
        "I messaggi inviati a questo gruppo",
        "ha creato il gruppo",
        "ora è un amministratore",
        "è stato cambiato con",
        "è entrato usando il link d'invito al gruppo",
        "ha eliminato l'immagine del gruppo"
    ]
    ANNUNCI = "Annunci WhatsApp"
    # Logica di parsing della linea di chat
    def __init__(self, line):
        try:
            # I messaggi sono del tipo "<data>, <ora> - <persona>: <messaggio>"
            # o del tipo "<data>, <ora> - <Annuncio Whatsapp>"
            stripped = line.split("-", 1)
            strippedm = stripped[1].split(":", 1)
            self.mittente = strippedm[0].strip()
            self.date = datetime.strptime(stripped[0], '%d/%m/%y, %H:%M ')
            # caso "<data>, <ora> - <Annuncio Whatsapp>"
            if len(strippedm) == 1 and any(s in line for s in self.messaggiWhatsapp):
                self.mittente = self.ANNUNCI
                self.messaggio = stripped[1].strip()
                
            # caso "<data>, <ora> - <persona>: <messaggio>"
            else:   
                self.mittente = strippedm[0].strip()
                # L'annuncio conteneva due punti, caso '<data>, <ora> - Hai cambiato l'oggetto in "qualcosa: titolo"'
                if any(s in self.mittente for s in self.messaggiWhatsapp):
                    self.mittente = self.ANNUNCI
                    self.messaggio = stripped[1].strip()
                else: 
                    self.messaggio = strippedm[1].strip()
            if not hasattr(self, "messaggio"):
                print "ERRORE nella seguente linea di chat (mandatamela come issue):"
                print line
        except:
            print "ERRORE nella seguente linea di chat (mandatamela come issue):"
            print line
    

# Importo i messaggi

In [ ]:
# Leggo i messaggi e li associo ad un oggetto 
regex=re.compile('^\d{2}\/\d{2}\/\d{2}\,\s\d{2}\:\d{2}')
with open(filename) as f:
    rawChat = []
    i = 0
    for line in f.read().split("\n"):
        if re.match(regex, line):
            rawChat.append(line)
            i += 1
        else:
            rawChat[i-1] += line

rawChat = [ChatLine(x) for x in rawChat]
chat = []
previous = None
# (messaggi di una stessa persona inviati di seguito sono considerati come un unico messaggio)
for mess in rawChat:
    if (previous is not None and previous.mittente == mess.mittente):
        previous.messaggio += mess.messaggio
    else:
        chat.append(mess)
        previous = mess

# Analisi generica dei dati ottenuti

In [ ]:
dirName = "risultati"
if not os.path.exists(dirName):
    os.makedirs(dirName)

data = {}
data["messaggi_inviati"] = {}
for mess in chat:
    if mess.mittente == ChatLine.ANNUNCI:
        continue
    if not mess.mittente in data["messaggi_inviati"]:
        data["messaggi_inviati"][mess.mittente] = 0
    data["messaggi_inviati"][mess.mittente] += 1
        
data["link_inviati"] = {}
for mess in chat:
    if mess.mittente == ChatLine.ANNUNCI:
        continue
    if not mess.mittente in data["link_inviati"]:
        data["link_inviati"][mess.mittente] = 0
    if ("http" in mess.messaggio):
        data["link_inviati"][mess.mittente] += 1
data["link_per_messaggio"] = {}
for f in data["link_inviati"]:
    data["link_per_messaggio"][f] = data["link_inviati"][f] / (data["messaggi_inviati"][f] * 1.0)
    
data["media_inviati"] = {}
for mess in chat:
    if mess.mittente == ChatLine.ANNUNCI:
        continue
    if not mess.mittente in data["media_inviati"]:
        data["media_inviati"][mess.mittente] = 0
    if ("<Media omesso>" in mess.messaggio):
        data["media_inviati"][mess.mittente] += 1
        
data["media_inviati_per_messaggio"] = {}
for f in data["media_inviati"]:
    data["media_inviati_per_messaggio"][f] = data["media_inviati"][f] / (data["messaggi_inviati"][f] * 1.0)
    
df = pd.DataFrame(data)
df["messaggi_inviati"].plot(kind='pie',figsize=(13,13),fontsize=15, title="Messaggi inviati")
plt.axis('off')
plt.savefig(dirName + '/messaggi_inviati.png', dpi=300, bbox_inches='tight')
plt.show()

df["link_per_messaggio"].plot(kind='bar',figsize=(17,9),fontsize=15, title="Link per messaggio", grid=True)
plt.savefig(dirName + '/link_per_messaggio.png', dpi=300, bbox_inches='tight')
plt.show()

df["media_inviati_per_messaggio"].plot(kind='bar',figsize=(17,9),fontsize=15, title="Media inviati per messaggio", grid=True)
plt.savefig(dirName + '/media_inviati_per_messaggio.png', dpi=300, bbox_inches='tight')
plt.show()

# Analisi temporale

In [ ]:
calendario = {}

# Analizza i messaggi inviati ogni giorno
for mess in chat:
    if mess.mittente == ChatLine.ANNUNCI:
        continue
    date = mess.date.strftime('%Y/%m/%d')
    if not date in calendario:
        calendario[date] = 0
    calendario[date] += 1
    
df = pd.Series(calendario, name='DateValue')
df.plot(kind='line',figsize=(55,9),fontsize=15, title="Messaggi inviati nel corso del tempo", grid="True", color="g")
plt.savefig(dirName + '/calendario.png', dpi=300, bbox_inches='tight')

ore = {}

for mess in chat:
    if mess.mittente == ChatLine.ANNUNCI:
        continue
    date = mess.date.strftime('%H')
    if not mess.mittente in ore:
        ore[mess.mittente] = {}
    if not date in ore[mess.mittente]:
        ore[mess.mittente][date] = 0
    ore[mess.mittente][date] += 1
    
df = pd.DataFrame(ore)
df.fillna(0, inplace=True)
df.plot(kind='line',figsize=(15,9),fontsize=15, title="Frequenza oraria di utilizzo", grid="True")
plt.savefig(dirName + '/uso_orario.png', dpi=300, bbox_inches='tight')


# Analisi parole più utilizzate

In [ ]:
reload(sys)
sys.setdefaultencoding('utf-8')
# Parole escluse (Parole più comuni)
escluse = open("topParole.txt").read().split("\n")
escluse = map(lambda x: x.lower(), escluse)

words = {}
for mess in chat:
    if mess.mittente == ChatLine.ANNUNCI:
        continue
    messaggio = mess.messaggio
    parole = messaggio.split(" ")
    parole = map(lambda x: x.lower(), parole)
    for parola in parole:
        if parola not in words:
            words[parola] = 0
        if len(parola) > 3 and parola not in escluse:
            words[parola] += 1
            
df = pd.Series(words, name="Parole più utilizzate")
df = df.to_frame()
df = df.sort_values(['Parole più utilizzate'], ascending=[0])
# Vogliamo solo le prime 20 parole
df = df.head(20)
df.plot(kind='bar',figsize=(25,7),fontsize=15, \
        title="Parole più utilizzate (escluse le parole piu comuni della lingua italiana e quelle più corte di 3 lettere)", grid=True)
plt.savefig(dirName + '/uso_parole.png', dpi=300, bbox_inches='tight')